# Keybert + TF-IDF

In [143]:
import pandas as pd
import numpy as np

# 정규식 활용 -> 특수문자 제거
import re

# 한글 형태소 분석기
from konlpy.tag import Okt
okt = Okt()

# keybert
from keybert import KeyBERT

In [8]:
doc = """
카카오뱅크 제공  인터넷은행 카카오뱅크는 지난 2분기 말 개인사업자 대상 대출 잔액이 2조5000억원을 돌파했다고 8일 발표했다. 카카오뱅크가 개인사업자를 대상으로 대출을 처음 내주기 시작한 2022년 11월 이후 약 3년 만에 이룬 성과다. 카카오뱅크가 그동안 개인사업자에게 공급한 누적 대출액은 4조원에 육박했다. 카카오뱅크는 향후 개인사업자 대상 대출 포트폴리오를 다양화해 자영업자 소상공인 대상 포용금융을 강화할 계획이다.카카오뱅크가 현재 개인사업자를 대상으로 판매하고 있는 대출 상품은  개인사업자 신용대출 과  개인사업자 보증서대출  등 두 가지다. 2022년 11월 출시된 개인사업자 신용대출의 누적 공급액은 지난 7월 말 1조8500원으로 집계됐다. 개인사업자 보증서대출은 2023년 5월 출시된 이후 불과 2년 만인 지난 7월 말 1조9200억원이 누적 공급됐다. 두 상품을 합치면 누적 공급액이 지난 7월 말 기준 약 4조원에 달한다.대출 잔액 역시 꾸준히 증가하고 있다. 카카오뱅크의 지난 6월 말  개인사업자 신용대출 의 잔액은 1조원   개인사업자 보증서대출  잔액은 1조5000억원으로 집계됐다. 두 상품을 합치면 6월 말 잔액은 2조5000억원을 넘는다.최근 은행권이 건전성 관리를 위해 전반적으로 자영업자 소상공인 대사 대출을 줄이고 있는 것과는 다른 행보다. 카카오뱅크는  개인사업자 대출 잔액을 지난 1년 간 80% 넘게 늘리며 어려운 시기에 힘이 되는 포용금융을 실천하고 있다 고 강조했다.카카오뱅크는 향후 개인사업자 대상 대출 상품을 다양화할 계획이다. 우선 올 연말까지  개인사업자 담보대출 을 새로 출시할 예정이다. 개인사업자 담보대출은 신용대출보다 금리가 낮고 한도가 높게 책정될 가능성이 높아 자금이 필요한 자영업자 소상공인이 다양한 대출 선택지를 확보할 수 있을 전망이다.카카오뱅크의 개인사업자 대출 고객 가운데 중 저신용자가 차지하는 비중은 지난 7월 말 기준 64%로 집계됐다. 특히  개인사업자 보증서대출 의 경우 중 저신용 고객 비중이 더 높았다는 게 카카오뱅크의 설명이다.카카오뱅크는 보증서대출 고객을 대상으로 보증료와 이차보전 지원을 통해 금융비용 부담을 낮추고 있다. 2023년 5월 개인사업자 대상 보증서대출 출시 이후 보증료의 최대 절반을 지속 지원해 지난 7월 말까지 총 211억 원의 보증료를 대신 지급했다. 이에 따라 보증서대출을 실행한 개인사업자 고객 1인당 약 26만원의 보증료 절감 혜택을 봤다고 카카오뱅크는 설명했다.카카오뱅크를 통해  이자지원 보증서대출 을 받은 개인사업자 고객은 평균 연 2.26%라는 낮은 금리로 대출을 받았다. 카카오뱅크 고객들은 이차보전 지원 혜택을 받아 지난 7월 말 기준 약 2.25%의 금리를 절감했기 때문이다. 이차보전 지원을 통해 고객이 받은 이자 절감액은 누적 223억원에 달한다.카카오뱅크는 보증서대출 상품 종류도 지속 확장하고 있다. 작년 4월엔 인터넷은행 최초로 비대면 이차보전 상품을 출시했고  지난 3월에는 금융권 최초로 마이너스통장 형식의 보증서대출을 선보였다.카카오뱅크는 소상공인들을 위한 편의서비스도 다양화하고 있다. 지난 1월 선보인  부가세박스 는 개인사업자들이 사업 운영 중 발생하는 부가세를 미리 저축할 수 있도록 돕는 사업자 전용 상품이다. 연 2회의 부가세 납부 일정에 맞춰 자금을 마련할 수 있게  자동 모으기  및  부가세 리포트  기능을 제공한다.지난해 4월엔 금융권 최초로  고객 맞춤형 정책자금 대출상품 통합조회 서비스 를 출시했다. 단 한 번의 조회로 고객별로 신청 가능한 보증서대출을 모두 확인하도록 돕는 서비스다.카카오뱅크는  120만 개인사업자의  금융 파트너 로서 어려움을 겪는 소상공인들을 위한 금융 지원에 적극 동참하고 있다 며  앞으로도 소상공인의 금융 부담을 덜어주는 실질적인 지원과 편의 서비스로 포용금융을 실천하겠다 고 전했다.정의진 기자 justjin hankyung.com
"""

In [12]:
kw_model = KeyBERT()

# keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(2,4),
#                                      use_maxsum = True,top_n = 20)

keywords_mmr = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2,4),
                                         use_mmr = True, top_n = 10, diversity = 0.3)
keywords_mmr

[('보증서대출을 선보였다 카카오뱅크는 소상공인들을', 0.7174),
 ('전망이다 카카오뱅크의 개인사업자 대출', 0.7168),
 ('카카오뱅크는 120만 개인사업자의 금융', 0.7144),
 ('형식의 보증서대출을 선보였다 카카오뱅크는', 0.7144),
 ('카카오뱅크의 개인사업자 대출 고객', 0.7127),
 ('설명이다 카카오뱅크는 보증서대출 고객을', 0.7126),
 ('카카오뱅크의 설명이다 카카오뱅크는 보증서대출', 0.7106),
 ('확보할 있을 전망이다 카카오뱅크의', 0.7078),
 ('성과다 카카오뱅크가 그동안 개인사업자에게', 0.7048),
 ('누적 223억원에 달한다 카카오뱅크는', 0.6541)]

In [91]:
# (문구, 점수) 튜플 → 문구만 뽑아 합치기
merged_text = ' '.join(phrase for phrase, _ in keywords_mmr)
merged_text

'보증서대출을 선보였다 카카오뱅크는 소상공인들을 전망이다 카카오뱅크의 개인사업자 대출 카카오뱅크는 120만 개인사업자의 금융 형식의 보증서대출을 선보였다 카카오뱅크는 카카오뱅크의 개인사업자 대출 고객 설명이다 카카오뱅크는 보증서대출 고객을 카카오뱅크의 설명이다 카카오뱅크는 보증서대출 확보할 있을 전망이다 카카오뱅크의 성과다 카카오뱅크가 그동안 개인사업자에게 이룬 성과다 카카오뱅크가 그동안 전망이다 카카오뱅크의 개인사업자 설명이다 카카오뱅크는 보증서대출 비중이 높았다는 카카오뱅크의 설명이다 서비스다 카카오뱅크는 120만 개인사업자의 카카오뱅크는 보증서대출 고객을 대상으로 카카오뱅크를 통해 이자지원 보증서대출 카카오뱅크는 보증서대출 상품 종류도 카카오뱅크는 개인사업자 대출 잔액을 달한다 카카오뱅크는 보증서대출 상품 누적 223억원에 달한다 카카오뱅크는'

In [96]:
# 형태소 분석기 생성
okt = Okt()

# 토큰화
tokens = okt.morphs(merged_text)
tokens

# 불용어 불러오기
with open("/Users/leesangwon/Documents/ThemeStock_file/stopwords_kor.txt", "r", encoding="utf-8") as f:
    stopwords = f.read()

# 불용어 제거
tokens = [word for word in tokens if not word in stopwords]
tokens = " ".join(tokens)

# 다시 토큰화
cleaned_tokens = okt.morphs(tokens)
print(cleaned_tokens)

['보증', '대출', '카카오', '뱅크', '소상', '공인', '카카오', '뱅크', '개인', '사업자', '대출', '카카오', '뱅크', '120만', '개인', '사업자', '금융', '보증', '대출', '카카오', '뱅크', '카카오', '뱅크', '개인', '사업자', '대출', '고객', '설명', '카카오', '뱅크', '보증', '대출', '고객', '카카오', '뱅크', '설명', '카카오', '뱅크', '보증', '대출', '확보', '있을', '카카오', '뱅크', '성과', '카카오', '뱅크', '개인', '사업자', '이룬', '성과', '카카오', '뱅크', '카카오', '뱅크', '개인', '사업자', '설명', '카카오', '뱅크', '보증', '대출', '비중', '높았다는', '카카오', '뱅크', '설명', '서비스', '카카오', '뱅크', '120만', '개인', '사업자', '카카오', '뱅크', '보증', '대출', '고객', '대상', '카카오', '뱅크', '이자지', '원', '보증', '대출', '카카오', '뱅크', '보증', '대출', '상품', '종류', '카카오', '뱅크', '개인', '사업자', '대출', '잔액', '카카오', '뱅크', '보증', '대출', '상품', '누적', '223억원', '카카오', '뱅크']


In [97]:
from collections import Counter

# 1. 빈도 계산
word_counts = Counter(cleaned_tokens)

# 2. DataFrame 변환
df_counts = pd.DataFrame(word_counts.items(), columns=['word', 'count'])

# 3. count 기준 내림차순 정렬
df_counts = df_counts.sort_values(by='count', ascending=False).reset_index(drop=True)

df_counts.head(50)

,word,count
0,카카오,21
1,뱅크,21
2,대출,12
3,보증,9
4,개인,7
5,사업자,7
6,설명,4
7,고객,3
8,120만,2
9,성과,2
